In [ ]:
# Install vertica driver
# There are different options here

# Formal native HP driver - Didn't support BULK LOAD
# pip install /opt/vertica/Python/vertica-db-client-8.1.1.0-cp27-none-linux_x86_64.whl

# ODBC Driver
# pip install pyodbc

# Native python driver - Mantained by UBER
!pip install vertica_python



# BULK LOAD IN VERTICA

In [ ]:
# Load CSV in Vertica
import csv

# Check file
with open("/home/datascience/data/Uber-Jan-Feb-FOIL.csv", "rb") as fs:
    reader = csv.reader(fs)
    headers = reader.next()
    data = reader.next()
print(headers)
print(data)


In [ ]:
import vertica_python

conn_info = {'host': '127.0.0.1',
             'port': 5433,
             'user': 'dbadmin',
             'password': '',
             'database': 'docker',
             # 10 minutes timeout on queries
             'read_timeout': 600,
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': False,
             # connection timeout is not enabled by default
             'connection_timeout': 5             
            }

# Connect to Vertica
db = vertica_python.connect(**conn_info)

# Create table
cur = db.cursor()
cur.execute("DROP TABLE IF EXISTS Uber_Users")
# For simplicity we assume all columns are varchar
cur.execute("CREATE TABLE Uber_Users (" + ", ".join([col_name + " varchar" for col_name in headers]) + ")")

# Load Data
with open("/home/datascience/data/Uber-Jan-Feb-FOIL.csv", "rb") as fs:
    # Skip first row (contains the header)
    cur.copy("COPY Uber_Users FROM STDIN DELIMITER ','  SKIP 1", fs)

# Commit and close
db.commit()
db.close()

# Load data from Vertica

In [ ]:
import vertica_python
import pandas as pd

conn_info = {'host': '127.0.0.1',
             'port': 5433,
             'user': 'dbadmin',
             'password': '',
             'database': 'docker',
             # 10 minutes timeout on queries
             'read_timeout': 600,
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': False,
             # connection timeout is not enabled by default
             'connection_timeout': 5             
            }

# Connect to Vertica
db = vertica_python.connect(**conn_info)

cur = db.cursor() # Return results as list
# cur = db.cursor('dict') # Return results as dictionary
cur.execute("SELECT * FROM Uber_Users;") # Run query
uber_data = cur.fetchall() # Collect results

uber_cols = [column[0] for column in cur.description] # Get columns name

print(uber_cols)
print(uber_data[0:10])

In [ ]:
# Alternatively you can load data with Pandas

import vertica_python
import pandas as pd

conn_info = {'host': '127.0.0.1',
             'port': 5433,
             'user': 'dbadmin',
             'password': '',
             'database': 'docker',
             # 10 minutes timeout on queries
             'read_timeout': 600,
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': False,
             # connection timeout is not enabled by default
             'connection_timeout': 5             
            }

# Connect to Vertica
db = vertica_python.connect(**conn_info)

uber_data = pd.read_sql("SELECT * FROM Uber_Users;", db)
uber_data

# Load Data in SPARK

In [ ]:
import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)

opts={}
opts['table']='Uber_Users'
opts['db']='docker'
opts['user']='dbadmin'
opts['password']=''
opts['host']='127.0.0.1'

uber_data = sqlContext.read.load(format="com.vertica.spark.datasource.DefaultSource", **opts)


In [ ]:
uber_data.show()